In [16]:
import pandas as pd
import numpy as np
import pickle
import os
import glob
from tqdm import tqdm


In [3]:
# constants
data_dir = 'data/spacenet/'
AOI = 'AOI_2_Vegas_Train/'


image_prefix = 'RGB-PanSharpen_' # RGB-PanSharpen_AOI_2_Vegas_img1.tif
geojson_prefix = 'buildings_' # buildings_AOI_2_Vegas_img1.geojson

# image_dir = os.path.join(data_dir, AOI, 'RGB-PanSharpen')
image_dir = os.path.join(data_dir, AOI, 'test_RGB-PanSharpen')
geojson_dir = os.path.join(data_dir, AOI, 'geojson', 'buildings')
output_dir = os.path.join(data_dir, AOI, 'RGB-PanSharpen_masks')



In [28]:
summary_csv = os.path.join(data_dir, AOI, 'summaryData', 'AOI_2_Vegas_Train_Building_Solutions.csv')
summary_csv


'data/spacenet/AOI_2_Vegas_Train/summaryData/AOI_2_Vegas_Train_Building_Solutions.csv'

In [29]:
summary_csv = os.path.join(data_dir, AOI, 'summaryData', 'AOI_2_Vegas_Train_Building_Solutions.csv')
# summary_csv = 'data/spacenet/AOI_2_Vegas_Train/summaryData/AOI_2_Vegas_Train_Building_Solutions.csv'
df = pd.read_csv(summary_csv)

In [30]:
df.head(2)

,ImageId,BuildingId,PolygonWKT_Pix,PolygonWKT_Geo
0,AOI_2_Vegas_img1209,1,"POLYGON ((68.71 623.1 0,35.71 618.15 0,33.59 6...","POLYGON ((-115.27398708 36.20208033800003 0,-1..."
1,AOI_2_Vegas_img1209,2,"POLYGON ((221.33 645.02 0,185.7 640.33 0,184.4...",POLYGON ((-115.273575016999985 36.202021143000...


In [14]:
image_ids = df['ImageId'].unique()

In [32]:
sorted(image_ids).shape

AttributeError: 'list' object has no attribute 'shape'

In [17]:
for index, image_id in tqdm(enumerate(image_ids)) :
    if index > 10:
        break
    print(image_id)

0it [00:00, ?it/s]

AOI_2_Vegas_img1209
AOI_2_Vegas_img5289
AOI_2_Vegas_img2917
AOI_2_Vegas_img2241
AOI_2_Vegas_img3543
AOI_2_Vegas_img4703
AOI_2_Vegas_img1257
AOI_2_Vegas_img1898
AOI_2_Vegas_img1233
AOI_2_Vegas_img327
AOI_2_Vegas_img2954
AOI_2_Vegas_img3044
AOI_2_Vegas_img3325
AOI_2_Vegas_img4742
AOI_2_Vegas_img2990
AOI_2_Vegas_img149
AOI_2_Vegas_img2264
AOI_2_Vegas_img127
AOI_2_Vegas_img5860
AOI_2_Vegas_img1893
AOI_2_Vegas_img2245
